In [1]:
import pandas as pd
import sqlite3

## create a connection to the database using the library sqlite3

In [2]:
connection = sqlite3.connect('../../data/checking-logs.sqlite')

## using only one query for each of the groups, create two dataframes: test_results and control_results with the columns time and avg_diff and only two rows
  * time should have the values: after and before
  * avg_diff contains the average delta among all the users for the time period
  before each of them made their first visit to the page and afterward
  * only take into account the users that have observations before and after

In [7]:
test_results = pd.io.sql.read_sql("SELECT CASE WHEN first_commit_ts < first_view_ts THEN 'before' ELSE 'after' END AS time, " \
"AVG(diff) FROM "
"(SELECT t.uid, t.labname, t.first_commit_ts, t.first_view_ts, CAST((-julianday(datetime(d.deadlines, 'unixepoch')) + julianday(t.first_commit_ts))*24 AS INTEGER) AS diff FROM test as t " \
"JOIN deadlines as d " \
"ON d.labs = t.labname " \
"WHERE t.labname != 'project1')" \
"GROUP BY time", connection)
test_results

,time,AVG(diff)
0,after,-103.40625
1,before,-60.56250


In [8]:
control_results = pd.io.sql.read_sql("SELECT CASE WHEN first_commit_ts < first_view_ts THEN 'before' ELSE 'after' END AS time, " \
"AVG(diff) FROM "
"(SELECT t.uid, t.labname, t.first_commit_ts, t.first_view_ts, CAST((-julianday(datetime(d.deadlines, 'unixepoch')) + julianday(t.first_commit_ts))*24 AS INTEGER) AS diff FROM control as t " \
"JOIN deadlines as d " \
"ON d.labs = t.labname " \
"WHERE t.labname != 'project1')" \
"GROUP BY time", connection)
control_results

,time,AVG(diff)
0,after,-112.710526
1,before,-99.464286


## close the connection

In [9]:
connection.close()

## have the answer: did the hypothesis turn out to be true and the page does affect the students’ behavior?
да, дельта изменилась для тест